# Приложение для сервиса чтения книг

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

## Подключение 

In [1]:
#импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
#устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
#сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

In [4]:
#создадим цикл и выведем первые 5 строк каждого датафрейма
datasets = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for i in datasets:
    query = f"SELECT * FROM {i}"
    data = pd.io.sql.read_sql(query, con = engine)
    print(data.head())
    print('---------------------------------------------------------------')

   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  
---------------------------------------------------------------
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4           

### Задача №1 

Посчитайте, сколько книг вышло после 1 января 2000 года

In [5]:
task_one = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE publication_date > '2000-1-1';
        '''
pd.io.sql.read_sql(task_one, con = engine)

,count
0,819


С 1 января 2000 года вышло 819 книг

### Задача №2

Для каждой книги посчитайте количество обзоров и среднюю оценку

In [6]:
task_two = '''
            SELECT b.title,
                b.book_id,
                COUNT(DISTINCT r.review_id) AS reviews_count,
                ROUND(AVG(ra.rating),2) AS avg_rating
            FROM books AS b
            LEFT JOIN reviews AS r ON b.book_id = r.book_id
            LEFT JOIN ratings AS ra ON ra.book_id = b.book_id
            GROUP BY b.book_id;
            '''
pd.io.sql.read_sql(task_two, con = engine)

,title,book_id,reviews_count,avg_rating
0,'Salem's Lot,1,2,3.67
1,1 000 Places to See Before You Die,2,1,2.50
2,13 Little Blue Envelopes (Little Blue Envelope...,3,3,4.67
3,1491: New Revelations of the Americas Before C...,4,2,4.50
4,1776,5,4,4.00
...,...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),996,3,3.67
996,Xenocide (Ender's Saga #3),997,3,3.40
997,Year of Wonders,998,4,3.20
998,You Suck (A Love Story #2),999,2,4.50


### Задача №3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [7]:
task_three = '''
            SELECT p.publisher,
                p.publisher_id,
                COUNT(b.book_id) AS book_cnt
            FROM publishers AS p
            LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY book_cnt DESC
            LIMIT 1;
            '''
pd.io.sql.read_sql(task_three, con = engine)

,publisher,publisher_id,book_cnt
0,Penguin Books,212,42


Издательство "Penguin Books" выпустило 42 книги толще 50 страниц

### Задача №4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [8]:
task_four = '''
        SELECT a.author,
            ROUND(AVG(ra.rating),2) AS avg_rate
        FROM  ratings AS ra
        JOIN books AS b ON ra.book_id = b.book_id
        JOIN authors AS a ON a.author_id = b.author_id
        WHERE b.book_id IN(SELECT book_id
                            FROM ratings 
                            GROUP BY book_id
                            HAVING COUNT(rating) >= 50)
        GROUP BY a.author_id
        ORDER BY avg_rate DESC
        LIMIT 1;
            '''
pd.io.sql.read_sql(task_four, con = engine)

,author,avg_rate
0,J.K. Rowling/Mary GrandPré,4.29


Автором с самой высокой средней оценкой книг (4.29) является J.K. Rowling в соавторстве с Mary GrandPré

### Задача №5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [9]:
task_five = '''
            SELECT AVG(i.cnt_review) as avg_review
            FROM (
                SELECT COUNT(review_id) as cnt_review
                FROM reviews
                WHERE username IN (
                            SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(ratings) > 48)
                GROUP BY username) AS i
            '''
pd.io.sql.read_sql(task_five, con = engine)

,avg_review
0,24.0


Среднее количество обзоров от пользователей, поставивших более 48 оценок равно 24

## Заключение

В результате проведения исследования:
- было произведено подключение к БД
- исследованы таблицы
- рассчитаны 5 задач